## Final Project Submission

Please fill out:
* Student name: Catherine Gachiri
* Student pace: Remote
* Scheduled project review date/time: 14/09/2025
* Instructor name: Fidelis Wanalwenge
* Blog post URL:


# 🎬 Movie Studio Investment Analysis

## Project Overview
This notebook explores movie performance data to help our company decide **what types of films to create**.  
We will use exploratory data analysis and statistical modeling to answer business questions about ROI.

**Key Data Sources:**
- **The Numbers** (`tn.movie_budgets.csv.gz`) → Budgets & grosses (used for ROI).
- **IMDB** (`im.db`) → Movie metadata (genres, runtime, year).
- **Box Office Mojo** (`bom.movie_gross.csv.gz`) → Additional grosses (optional).

**Goal:** Build a dataset that combines **financial data** (budgets & grosses) with **metadata** (genres, runtime, release timing) for statistical analysis.

## Business Understanding
Our stakeholders (head of the new movie studio) want to know:

1. **Genres vs ROI** – Which genres yield the best returns?  
2. **Release Quarter vs ROI** – Does timing affect financial success?  
3. **Budget vs ROI %** – Are bigger budgets more (or less) profitable?  
4. **Runtime vs ROI** – Does movie length impact profitability?

We will prepare a clean dataset to test these hypotheses.

## Step 1: Load and Inspect Data

In [1]:
import pandas as pd
import numpy as np
import sqlite3
import re
from pathlib import Path

# Define paths
data_dir = Path('zippedData')
bom_path = data_dir/'bom.movie_gross.csv.gz'
tn_path = data_dir/'tn.movie_budgets.csv.gz'
imdb_path = Path('zippedData/im.db')

# Load Box Office Mojo
bom = pd.read_csv(bom_path)
print("Box Office Mojo sample:")
display(bom.head())
display(bom.shape)

# Load The Numbers (budgets)
tn = pd.read_csv(tn_path)
print("The Numbers sample:")
display(tn.head())
display(tn.shape)

# Inspect IMDB tables
con = sqlite3.connect(imdb_path)
tables = pd.read_sql("SELECT name FROM sqlite_master WHERE type='table';", con)
print("IMDB Tables:")
display(tables)


Box Office Mojo sample:


,title,studio,domestic_gross,foreign_gross,year
0,Toy Story 3,BV,415000000.0,652000000,2010
1,Alice in Wonderland (2010),BV,334200000.0,691300000,2010
2,Harry Potter and the Deathly Hallows Part 1,WB,296000000.0,664300000,2010
3,Inception,WB,292600000.0,535700000,2010
4,Shrek Forever After,P/DW,238700000.0,513900000,2010


(3387, 5)

The Numbers sample:


,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"
2,3,"Jun 7, 2019",Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350"
3,4,"May 1, 2015",Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963"
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747"


(5782, 6)

IMDB Tables:


,name
0,movie_basics
1,directors
2,known_for
3,movie_akas
4,movie_ratings
5,persons
6,principals
7,writers
8,movies_financials_ratings
9,merged_with_ratings


## Step 2: Clean `The Numbers` Dataset

In [2]:
tn_clean = tn.copy()

# Convert currency columns to numeric
currency_cols = ["production_budget", "domestic_gross", "worldwide_gross"]
for col in currency_cols:
    tn_clean[col] = (tn_clean[col]
                     .replace('[\$,]', '', regex=True)
                     .astype(float))

# Parse release date
tn_clean["release_date"] = pd.to_datetime(tn_clean["release_date"], errors="coerce")
tn_clean["year"] = tn_clean["release_date"].dt.year
tn_clean["quarter"] = tn_clean["release_date"].dt.quarter

# Compute ROI
tn_clean["ROI"] = (tn_clean["worldwide_gross"] - tn_clean["production_budget"]) / tn_clean["production_budget"]

display(tn_clean.head())
display(tn_clean.shape)

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross,year,quarter,ROI
0,1,2009-12-18,Avatar,425000000.0,760507625.0,2.776345e+09,2009,4,5.532577
1,2,2011-05-20,Pirates of the Caribbean: On Stranger Tides,410600000.0,241063875.0,1.045664e+09,2011,2,1.546673
2,3,2019-06-07,Dark Phoenix,350000000.0,42762350.0,1.497624e+08,2019,2,-0.572108
3,4,2015-05-01,Avengers: Age of Ultron,330600000.0,459005868.0,1.403014e+09,2015,2,3.243841
4,5,2017-12-15,Star Wars Ep. VIII: The Last Jedi,317000000.0,620181382.0,1.316722e+09,2017,4,3.153696


(5782, 9)

## Step 3: Extract Metadata from IMDB

In [3]:
con = sqlite3.connect(imdb_path)
imdb = pd.read_sql("""
    SELECT movie_id, primary_title, start_year, runtime_minutes, genres
    FROM movie_basics
    WHERE start_year BETWEEN 1980 AND 2025
      AND primary_title IS NOT NULL
""", con)
con.close()

imdb.head()


,movie_id,primary_title,start_year,runtime_minutes,genres
0,tt0063540,Sunghursh,2013,175.0,"Action,Crime,Drama"
1,tt0066787,One Day Before the Rainy Season,2019,114.0,"Biography,Drama"
2,tt0069049,The Other Side of the Wind,2018,122.0,Drama
3,tt0069204,Sabse Bada Sukh,2018,NaN,"Comedy,Drama"
4,tt0100275,The Wandering Soap Opera,2017,80.0,"Comedy,Drama,Fantasy"


## Step 4: Normalize Titles and Join Datasets

In [4]:
# Title normalization function
def normalize_title(title: str) -> str:
    if pd.isna(title):
        return np.nan
    title = title.lower().strip()
    title = re.sub(r"\([^)]*\)", "", title)  # remove parentheticals
    title = re.sub(r"[^a-z0-9 ]", "", title)   # drop punctuation
    title = re.sub(r"\s+", " ", title).strip()
    return title

tn_clean["title_norm"] = tn_clean["movie"].map(normalize_title)
imdb["title_norm"] = imdb["primary_title"].map(normalize_title)

# Bring in ratings to get numvotes
con = sqlite3.connect(imdb_path)
ratings = pd.read_sql("SELECT movie_id, averagerating, numvotes FROM movie_ratings;", con)
con.close()

imdb_full = (imdb.merge(ratings, on='movie_id', how='left')
                 .assign(numvotes=lambda d: d['numvotes'].fillna(0),
                         runtime_minutes=lambda d: d['runtime_minutes'].fillna(-1)))

# Sort by best proxy for canonical record, then keep first per key
imdb_dedup = (imdb_full.sort_values(['title_norm','start_year','numvotes','runtime_minutes'],
                                    ascending=[True, True, False, False])
                        .drop_duplicates(['title_norm','start_year'], keep='first')
                        .drop(columns=['averagerating','numvotes']))  # keep if you need them

# Re-join with deduped IMDB
movies_dedup = tn_clean.merge(
    imdb_dedup, left_on=['title_norm','year'], right_on=['title_norm','start_year'], how='left'
)
len(movies_dedup) - len(tn_clean)   # ← should now be ~0 (or much smaller)

display(movies_dedup.head())
display(movies_dedup.shape)

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross,year,quarter,ROI,title_norm,movie_id,primary_title,start_year,runtime_minutes,genres
0,1,2009-12-18,Avatar,425000000.0,760507625.0,2.776345e+09,2009,4,5.532577,avatar,NaN,NaN,NaN,NaN,NaN
1,2,2011-05-20,Pirates of the Caribbean: On Stranger Tides,410600000.0,241063875.0,1.045664e+09,2011,2,1.546673,pirates of the caribbean on stranger tides,tt1298650,Pirates of the Caribbean: On Stranger Tides,2011.0,136.0,"Action,Adventure,Fantasy"
2,3,2019-06-07,Dark Phoenix,350000000.0,42762350.0,1.497624e+08,2019,2,-0.572108,dark phoenix,tt6565702,Dark Phoenix,2019.0,113.0,"Action,Adventure,Sci-Fi"
3,4,2015-05-01,Avengers: Age of Ultron,330600000.0,459005868.0,1.403014e+09,2015,2,3.243841,avengers age of ultron,tt2395427,Avengers: Age of Ultron,2015.0,141.0,"Action,Adventure,Sci-Fi"
4,5,2017-12-15,Star Wars Ep. VIII: The Last Jedi,317000000.0,620181382.0,1.316722e+09,2017,4,3.153696,star wars ep viii the last jedi,NaN,NaN,NaN,NaN,NaN


(5782, 15)

## Step 5: Create Final Analysis Dataset

In [5]:
analysis_df = movies_dedup[[
    "movie", "release_date", "year", "quarter",
    "production_budget", "worldwide_gross", "ROI",
    "runtime_minutes", "genres"
]].copy()

display(analysis_df.head(10))
analysis_df.shape

,movie,release_date,year,quarter,production_budget,worldwide_gross,ROI,runtime_minutes,genres
0,Avatar,2009-12-18,2009,4,425000000.0,2.776345e+09,5.532577,NaN,NaN
1,Pirates of the Caribbean: On Stranger Tides,2011-05-20,2011,2,410600000.0,1.045664e+09,1.546673,136.0,"Action,Adventure,Fantasy"
2,Dark Phoenix,2019-06-07,2019,2,350000000.0,1.497624e+08,-0.572108,113.0,"Action,Adventure,Sci-Fi"
3,Avengers: Age of Ultron,2015-05-01,2015,2,330600000.0,1.403014e+09,3.243841,141.0,"Action,Adventure,Sci-Fi"
4,Star Wars Ep. VIII: The Last Jedi,2017-12-15,2017,4,317000000.0,1.316722e+09,3.153696,NaN,NaN
5,Star Wars Ep. VII: The Force Awakens,2015-12-18,2015,4,306000000.0,2.053311e+09,5.710167,NaN,NaN
6,Avengers: Infinity War,2018-04-27,2018,2,300000000.0,2.048134e+09,5.827114,149.0,"Action,Adventure,Sci-Fi"
7,Pirates of the Caribbean: At Worldâs End,2007-05-24,2007,2,300000000.0,9.634204e+08,2.211401,NaN,NaN
8,Justice League,2017-11-17,2017,4,300000000.0,6.559452e+08,1.186484,120.0,"Action,Adventure,Fantasy"
9,Spectre,2015-11-06,2015,4,300000000.0,8.796209e+08,1.932070,148.0,"Action,Adventure,Thriller"


(5782, 9)

In [6]:
# Drop rows without ROI
df = analysis_df.dropna(subset=["ROI"]).copy()

# Extract primary genre (first listed)
#df["primary_genre"] = df["genres"].dropna().apply(lambda x: x.split(",")[0] if isinstance(x, str) else np.nan)

# 1) Explode genres so a movie appears once per genre
df_multi = df.dropna(subset=["genres"]).copy()
df_multi["genre"] = df_multi["genres"].str.split(",")
df_multi = df_multi.explode("genre")
df_multi["genre"] = df_multi["genre"].str.strip()

# 2) Cluster id: all repeated rows from same movie share this
# (use an actual unique id if you have it; title+year is a good fallback)
df_multi["cluster_id"] = (
    df_multi["movie"].str.lower().str.strip() + "_" + df_multi["year"].astype(str)
)

# (optional) keep genres with enough data
counts = df_multi["genre"].value_counts()
keep = counts[counts >= 30].index
sub = df_multi[df_multi["genre"].isin(keep)].copy()

# Hypothesis-specific datasets
df_genre   = df_multi.dropna(subset=["genre"])
df_quarter = df.dropna(subset=["quarter"])
df_budget  = df[df["production_budget"] > 0].copy()
df_runtime = df.dropna(subset=["runtime_minutes"])

print("Base dataset:", df.shape)
print("Genre dataset:", df_genre.shape)
print("Quarter dataset:", df_quarter.shape)
print("Budget dataset:", df_budget.shape)
print("Runtime dataset:", df_runtime.shape)

Base dataset: (5782, 9)
Genre dataset: (4030, 11)
Quarter dataset: (5782, 9)
Budget dataset: (5782, 9)
Runtime dataset: (1582, 9)


Test whether the length of a movie (runtime) has an effect on its profitability (ROI) using Simple Linear Regression Model. 

- Independent Variable (X): Runtime (minutes)


- Dependent Variable (Y): ROI (Return on Investment) 


- Null Hypothesis (H₀): There is no relationship between movie runtime and ROI


- Alternative Hypothesis (H₁): There is a relationship between movie runtime and ROI

In [7]:
# Preparation - Create a join key using title + year

tn_clean["title_key"] = tn_clean["movie"].str.lower().str.strip() + tn_clean["year"].astype(str)
imdb["title_key"] = imdb["primary_title"].str.lower().str.strip() + imdb["start_year"].astype(str)

runtime_data = imdb.merge(tn_clean, on="title_key", how="inner")

print(runtime_data[["primary_title", "runtime_minutes", "ROI"]].head())

                     primary_title  runtime_minutes       ROI
0                       Foodfight!             91.0 -0.998362
1  The Secret Life of Walter Mitty            114.0  1.064409
2      A Walk Among the Tombstones            114.0  1.218164
3                   Jurassic World            124.0  6.669092
4                    The Rum Diary            119.0 -0.521228


In [ ]:
import warnings
warnings.simplefilter(action="ignore", category=FutureWarning)

import statsmodels.api as sm

# Drop missing runtimes or ROIs
runtime_data = runtime_data.dropna(subset=["runtime_minutes", "ROI"])

# Define variables
X = runtime_data["runtime_minutes"]
y = runtime_data["ROI"]

# Add intercept
X = sm.add_constant(X)

# Fit regression
model = sm.OLS(y, X).fit()
print(model.summary())

# 95% CI for slope
print("95% CI for slope:", model.conf_int().loc["runtime_minutes"])


The R² value (0.001) means runtime explains almost nothing about how profitable a movie is.

The slope (-0.0144) and p-value (0.359) mean longer or shorter movies don’t significantly change profits.

The confidence interval includes zero, which means the effect could be slightly positive or negative, but overall it’s too small to matter.


In [ ]:
# Visualization

import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(10,7))
sns.regplot(x="runtime_minutes", y="ROI", data=runtime_data,
            scatter_kws={"alpha":0.5}, line_kws={"color":"red"})

plt.title("Runtime vs Return on Investment with Regression Line")
plt.xlabel("Runtime (minutes)")
plt.ylabel("Return on Investment")
plt.show()

The visualization displayed above is a scatter plot of movie runtime vs ROI, with the regression line (in red) drawn across the data points. This makes it clear that the trend line is almost flat, confirming that runtime has no meaningful effect on profitability.

- Null Hypothesis (H₀): There is no relationship between movie runtime and ROI.

- Alternative Hypothesis (H₁):There is a relationship between movie runtime and ROI.

- Since the p-value = 0.359 > 0.05, we fail to reject H₀ → meaning runtime does not significantly impact profitability.

### Business Recommendation

The analysis shows that movie length does not significantly impact profitability, whether a film runs shorter or longer has almost no effect on its return on investment (ROI).

#### Implication for the Company:

Runtime should not be a deciding factor when selecting or producing films.Strategic focus should shift to more influential drivers of success such as:

 - Budget management (spending efficiently to maximize returns)

 - Release timing (launching films in profitable quarters/seasons)

 - Marketing and distribution strategies

Advice to Stakeholder: When developing original video content do not prioritize on movie length.

